# 01 Example

Here is a starter Jupyter notebook to get started.


In [1]:
#neccessary imports
import pandas as pd
import pyarrow.dataset as ds
from loguru import logger
import wandb
import torch.nn as nn
import torch.nn.functional as F
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mayarahmed. Use `wandb login --relogin` to force relogin


True

In [10]:

path_train_data = "../../../data/train.parquet"
data_train = ds.dataset(source=path_train_data, format="parquet")

data_train.head(num_rows=5)


pyarrow.Table
id: int64
buildingblock1_smiles: string
buildingblock2_smiles: string
buildingblock3_smiles: string
molecule_smiles: string
protein_name: string
binds: int64
----
id: [[0,1,2,3,4]]
buildingblock1_smiles: [["C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21","C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21","C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21","C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21","C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21"]]
buildingblock2_smiles: [["C#CCOc1ccc(CN)cc1.Cl","C#CCOc1ccc(CN)cc1.Cl","C#CCOc1ccc(CN)cc1.Cl","C#CCOc1ccc(CN)cc1.Cl","C#CCOc1ccc(CN)cc1.Cl"]]
buildingblock3_smiles: [["Br.Br.NCC1CCCN1c1cccnn1","Br.Br.NCC1CCCN1c1cccnn1","Br.Br.NCC1CCCN1c1cccnn1","Br.NCc1cccc(Br)n1","Br.NCc1cccc(Br)n1"]]
molecule_smiles: [["C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1","C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1","C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H](CC#C)CC(=O)N[Dy])n2)cc1","C#

In [11]:
#filter using scanner to get the rows where the protein_name is BRD4
data_brd4 = data_train.scanner(filter=ds.field("protein_name") == "BRD4")

data_brd4=data_brd4.to_table().to_pandas()
#find the number of rows in the dataset
logger.info(f"Number of compounds targeting BRD4 protein {len(data_brd4)}")
#find the rows where the binds is 1
data_brd4_binds = data_brd4[data_brd4["binds"]==1]
logger.info(f"Number of compounds targeting BRD4 protein and binds is 1 {len(data_brd4_binds)}")


2024-06-24 22:46:39.006 | INFO     | __main__:<module>:6 - Number of compounds targeting BRD4 protein 98415610
2024-06-24 22:46:39.115 | INFO     | __main__:<module>:9 - Number of compounds targeting BRD4 protein and binds is 1 456964


In [12]:
#filter using scanner to get the rows where the protein_name is protein_name: HSA

data_hsa = data_train.scanner(filter=ds.field("protein_name") == "HSA")

data_hsa=data_hsa.to_table().to_pandas()
#find the number of rows in the dataset
logger.info(f"Number of compounds targeting HSA protein {len(data_hsa)}")
#find the rows where the binds is 1
data_hsa_binds = data_hsa[data_hsa["binds"]==1]
logger.info(f"Number of compounds targeting HSA protein and binds is 1 {len(data_hsa_binds)}")

2024-06-24 22:47:29.235 | INFO     | __main__:<module>:7 - Number of compounds targeting HSA protein 98415610
2024-06-24 22:47:29.470 | INFO     | __main__:<module>:10 - Number of compounds targeting HSA protein and binds is 1 408410


In [13]:
data_seh = data_train.scanner(filter=ds.field("protein_name") == "sEH")

data_seh=data_seh.to_table().to_pandas()
#find the number of rows in the dataset
logger.info(f"Number of compounds targeting sEH protein {len(data_seh)}")
#find the rows where the binds is 1
data_seh_binds = data_seh[data_seh["binds"]==1]
logger.info(f"Number of compounds targeting sEH protein and binds is 1 {len(data_seh_binds)}")

2024-06-24 22:48:19.724 | INFO     | __main__:<module>:5 - Number of compounds targeting sEH protein 98415610
2024-06-24 22:48:19.842 | INFO     | __main__:<module>:8 - Number of compounds targeting sEH protein and binds is 1 724532


In [14]:
#get from each protein a no bind subset that is equal to the number of bind subset
data_brd4_no_binds = data_brd4[data_brd4["binds"]==0].sample(n=len(data_seh_binds))
logger.info(f"Number of compounds targeting BRD4 protein and binds is 0 {len(data_brd4_no_binds)}")
data_hsa_no_binds = data_hsa[data_hsa["binds"]==0].sample(n=len(data_seh_binds))
logger.info(f"Number of compounds targeting HSA protein and binds is 0 {len(data_hsa_no_binds)}")
data_seh_no_binds = data_seh[data_seh["binds"]==0].sample(n=len(data_seh_binds))
logger.info(f"Number of compounds targeting sEH protein and binds is 0 {len(data_seh_no_binds)}")

2024-06-24 22:48:27.530 | INFO     | __main__:<module>:3 - Number of compounds targeting BRD4 protein and binds is 0 724532
2024-06-24 22:48:34.662 | INFO     | __main__:<module>:5 - Number of compounds targeting HSA protein and binds is 0 724532
2024-06-24 22:48:41.997 | INFO     | __main__:<module>:7 - Number of compounds targeting sEH protein and binds is 0 724532


In [15]:
# combine the dataframes into one and save it as a parquet file
data_combined = pd.concat([data_brd4_binds, data_hsa_binds, data_seh_binds, data_brd4_no_binds, data_hsa_no_binds, data_seh_no_binds])
data_combined.to_parquet("../../../data/train_combined.parquet")